# Automação de Sistemas e Processos com Python

### Desafio:

Diariamente, o nosso sistema atualiza as vendas do dia anterior.
O seu trabalho diário, como analista, é enviar um e-mail para a diretoria, assim que começar a trabalhar, com o faturamento e a quantidade de produtos vendidos no dia anterior

E-mail da diretoria: seugmail+diretoria@gmail.com<br>
Local onde o sistema disponibiliza as vendas do dia anterior: https://drive.google.com/drive/folders/149xknr9JvrlEnhNWO49zPcw0PW5icxga?usp=sharing

Para resolver isso, vamos usar o pyautogui, uma biblioteca de automação de comandos do mouse e do teclado

In [150]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import pandas as pd
import os
import pyautogui
import pyperclip

In [151]:
pyautogui.PAUSE = 1
DESTINATARIO = '4drade+diretoria@gmail.com'

In [152]:
# Passo 1: Entrar no sistema da empresa (no nosso caso é o link do drive)
# abrir navegador
navegador = webdriver.Chrome()
# abrir drive
navegador.get('https://drive.google.com/drive/folders/149xknr9JvrlEnhNWO49zPcw0PW5icxga?usp=sharing')

# Passo 2: Navegar no sistema e encontrar a base de vendas (entrar na pasta exportar)
# clicar na pasta
sleep(2)
navegador.find_element(By.XPATH, '//*[@id=":0"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div/c-wiz/div[1]/c-wiz[1]/c-wiz/div/c-wiz/div/div/div/div').click()
sleep(4)

# Passo 3: Fazer o download da base de vendas
# clicar no arquivo
navegador.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div[2]/div[2]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[2]').click()
sleep(5)
# baixar o arquivo
navegador.find_element(By.XPATH, '/html/body/div[11]/div[4]/div/div[3]/div[2]/div[2]/div[3]/div').click()
sleep(5)
# fechar navegador
navegador.quit()

### Vamos agora ler o arquivo baixado para pegar os indicadores

- Faturamento
- Quantidade de Produtos

In [153]:
# Passo 4: Importar a base de vendas para o Python
df = pd.read_excel(r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx')

In [154]:
# Passo 5: Calcular os indicadores
faturamento = df['Valor Final'].sum()
media_lucro = df['Valor Final'].mean()
qtd_produtos = df['Quantidade'].sum()
list_lojas = list(set(df['ID Loja']))
list_produtos = list(set(df['Produto']))
melhor = df.max()
pior = df.min()
produtos = [(df['Produto'][i], list(df['Produto']).count(list_produtos[i]))
            for i in range(len(list_produtos))]
lojas = [(df['ID Loja'][i], list(df['ID Loja']).count(list_lojas[i]))
         for i in range(len(list_lojas))]
mais_vendido = max(produtos, key=lambda p: p[1])
menos_vendido = min(produtos, key=lambda p: [1])
melhor_loja = max(lojas, key=lambda loja: loja[1])
pior_loja = min(lojas, key=lambda loja: loja[1])
hora_atual = datetime.now().hour

In [155]:
if hora_atual < 12:
    cumprimento = 'Bom dia'
elif hora_atual < 18:
    cumprimento = 'Boa tarde'
else:
    cumprimento = 'Boa noite'

### Vamos agora enviar um e-mail pelo gmail

In [156]:
# Passo 5: Entrar no email
# voltar ao navegador (clicar na tela)
pyautogui.click(826, y=129)
# abrir aba
pyautogui.hotkey('ctrl', 't')
# entrar no gmail
while not pyautogui.locateOnScreen('images/gmail.png'):
    sleep(0.5)
pyautogui.click(x=230, y=81)
# escrever e-mail
while not pyautogui.locateOnScreen('images/escrever.png'):
    sleep(0.5)
pyautogui.click(x=73, y=166)

# Passo 6: Digitar e enviar o resultado
while not pyautogui.locateOnScreen('images/mensagem.png'):
    sleep(1)

pyautogui.write(DESTINATARIO)
pyautogui.press('tab')  # seleciona o email

pyautogui.press('tab')  # pula para o campo de assunto
pyperclip.copy('Relatório de Vendas')
pyautogui.hotkey('ctrl', 'v')  # escreve o assunto
pyautogui.press('tab')  # pula para o corpo do email

texto = f'''Prezados, {cumprimento}

Segue relatório de vendas.
Faturamento total: R${faturamento:,.2f}
Média de faturamento por venda: R${media_lucro:,.2f}
Quantidade de produtos vendidos: {qtd_produtos:,}
Quantidade de lojas no momento: {len(list_lojas)}
Melhor venda:
    Data: {melhor['Data'].day}/{melhor['Data'].month}/{melhor['Data'].year}
    ID Loja: {melhor['ID Loja']}
    Produto: {melhor['Produto']}
    Quantidade: {melhor['Quantidade']:,}
    Valor Unitário: {melhor['Valor Unitário']:,.2f}
    Valor Final: R${melhor['Valor Final']:,.2f}
Pior venda:
    Data: {melhor['Data'].day}/{melhor['Data'].month}/{melhor['Data'].year}
    ID Loja: {pior['ID Loja']}
    Produto: {pior['Produto']}
    Quantidade: {pior['Quantidade']:,}
    Valor Unitário: {pior['Valor Unitário']:,.2f}
    Valor Final: R${pior['Valor Final']:,.2f}
Mais vendido:
    Nome: {mais_vendido[0]}
    Quantidade: {mais_vendido[1]}
Menos vendido:
    Nome: {menos_vendido[0]}
    Quantidade: {menos_vendido[1]}
Melhor loja:
    ID Loja: {melhor_loja[0]}
    Produtos vendidos: {melhor_loja[1]}
Pior loja:
    ID Loja: {pior_loja[0]}
    Produtos vendidos: {pior_loja[1]}

Qualquer dúvida estou à disposição.
Att.,
Felippe do Python'''

pyperclip.copy(texto)
pyautogui.hotkey('ctrl', 'v')

# anexar arquivo
pyautogui.click(x=950, y=706)
while not pyautogui.locateOnScreen('images/janela.png'):
    sleep(0.5)
pyperclip.copy(r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx')
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
while not pyautogui.locateOnScreen('images/arquivo_completo.png'):
    sleep(1)
pyautogui.hotkey('ctrl', 'enter')
while not pyautogui.locateOnScreen('images/mensagem_enviada.png'):
    sleep(1)
pyautogui.click(x=328, y=692)

In [157]:
try:
    os.remove(r'C:\Users\f4dra\Downloads\Vendas - Dez.xlsx')
    print('Arquivo removido.')
except FileNotFoundError:
    print('Arquivo não encontrado.')

Arquivo removido.
